In [1]:
import tensorflow as tf
tf.compat.v1.enable_v2_behavior()

/Users/tsukemm/Desktop/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tsukemm/Desktop/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tsukemm/Desktop/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tsukemm/Desktop/myenv/lib/python3.6/site-packages/tensorf

In [2]:
import tensorflow_datasets as tfds
first_50_percent = tfds.Split.TRAIN.subsplit(tfds.percent[:50])
last_50_percent = tfds.Split.TRAIN.subsplit(tfds.percent[-50:])

alice_dataset = tfds.load("fashion_mnist", split=first_50_percent)
bob_dataset = tfds.load("fashion_mnist", split=last_50_percent)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
import collections
BATCH_SIZE = 32

def cast(element):
    out = {}
    out["image"]=tf.image.convert_image_dtype(element['image'], dtype=tf.float32)
    out["label"]=element['label']
    return out

def flatten(element):
    return collections.OrderedDict([
        ('x', tf.reshape(element['image'], [-1])),
        ('y', tf.reshape(element['label'], [1])),
    ])

def preprocess(dataset):
    return dataset.map(cast).map(flatten).batch(BATCH_SIZE)

In [4]:
preprocessed_alice_dataset = preprocess(alice_dataset)
preprocessed_bob_dataset = preprocess(bob_dataset)

In [5]:
federated_data = [preprocessed_alice_dataset, preprocessed_bob_dataset]

In [6]:
from tensorflow.python.keras.optimizer_v2 import gradient_descent

LEARNING_RATE = 0.02
def custom_loss_function(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred))

def create_compiled_keras_model():
    model = tf.keras.models.Sequential([tf.keras.layers.Dense(10, activation=tf.nn.softmax, kernel_initializer='zeros', input_shape=(784,))])
    model.compile(loss=custom_loss_function, optimizer=gradient_descent.SGD(learning_rate=LEARNING_RATE), metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

In [7]:
batch_of_samples = tf.contrib.framework.nest.map_structure(lambda x: x.numpy(), iter(preprocessed_alice_dataset).next())
def model_instance():
    keras_model = create_compiled_keras_model()
    return tff.learning.from_compiled_keras_model(keras_model, batch_of_samples)

In [8]:
from tensorflow_federated import python as tff
federated_learning_iterative_process = tff.learning.build_federated_averaging_process(model_instance)
state = federated_learning_iterative_process.initialize()
state, performance = federated_learning_iterative_process.next(state, federated_data)

In [9]:
performance

AnonymousTuple([(sparse_categorical_accuracy, 0.74365), (loss, 0.82071316)])